Note: this notebook is set up to run with the env.yml containing the name 'polaris_datasets'

# Dataset creation with [Polaris](https://github.com/polaris-hub/polaris)
The first step of creating a benchmark is to set up a standard dataset which allows accessing the curated dataset (which has been demonstrated in <01_ncats_rlm_data_curation.ipynb>), and all necessary information about the dataset such as data source, description of endpoints, units etc. 

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm
import numpy as np

from polaris.cli import PolarisHubClient

# polaris dataset
from polaris.dataset import Dataset, ColumnAnnotation
from polaris.dataset._column import Modality

from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[3]
os.chdir(root)
sys.path.insert(0, str(root))
from utils.docs_utils import load_readme

In [2]:
# Get the owner and organization
org = "polaris"
data_name = "ncats_adme/RLM"
dirname = dm.fs.join(root, f"org-{org}", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"

owner = HubOwner(slug=org.lower(), type="organization")
owner

HubOwner(slug='polaris', external_id=None, type='organization')

In [3]:
BENCHMARK_DIR = f"{gcp_root}/benchmarks"
DATASET_DIR = f"{gcp_root}/datasets"
FIGURE_DIR =  f"{gcp_root}/figures"

## Load existing data

In [4]:
PATH = f'{gcp_root}/data/curation/{data_name}_curated.csv'
table = pd.read_csv(PATH)
table.columns

Index(['PUBCHEM_SID', 'SMILES', 'PUBCHEM_ACTIVITY_OUTCOME', 'PHENOTYPE',
       'HALF-LIFE', 'MOL_smiles', 'MOL_molhash_id', 'MOL_molhash_id_no_stereo',
       'MOL_num_stereoisomers', 'MOL_num_undefined_stereoisomers',
       'MOL_num_defined_stereo_center', 'MOL_num_undefined_stereo_center',
       'MOL_num_stereo_center', 'MOL_undefined_E_D', 'MOL_undefined_E/Z',
       'OUTLIER_PUBCHEM_ACTIVITY_OUTCOME', 'OUTLIER_PHENOTYPE',
       'OUTLIER_HALF-LIFE', 'AC_PUBCHEM_ACTIVITY_OUTCOME', 'AC_PHENOTYPE',
       'AC_HALF-LIFE'],
      dtype='object')

### Define `Dataset` object

In [5]:
version = "v1"
dataset_name = f"ncats_rlm-{version}"

### Below we specify the meta information of data columns

In [6]:
# Additional meta-data on the column level
annotations = {
    "MOL_molhash_id": ColumnAnnotation(
        description="Molecular hash ID. See <datamol.mol.hash_mol>"
    ),
    "MOL_smiles": ColumnAnnotation(
        description="Molecule SMILES string", modality="molecule"
    ),
    "PUBCHEM_ACTIVITY_OUTCOME": ColumnAnnotation(
        description="Binarized label based on the phenotype observed, active (class = 1) or inactive (class = 0)",
        user_attributes={"ref_col": "PHENOTYPE"},
    ),
    "PHENOTYPE": ColumnAnnotation(
        description="Indicates type of activity observed: Unstable (<30 min) (class = 0), Stable (>30 min) (class = 1).",
        user_attributes={"ref_col": "HALF-LIFE"},
    ),
    "HALF-LIFE": ColumnAnnotation(
        description="Rat liver microsome stability (T1/2)",  # We can use the metadata we collected previously
        user_attributes={"unit": "minutes"},
    ),
}

### Define `Dataset` object

In [7]:
version = "v1"
dataset_name = f"ncats_rlm-{version}"

In [8]:
dataset = Dataset(
    # The table is the core data-structure required to construct a dataset
    table=table[annotations.keys()], 
    # Additional meta-data on the dataset level.
    name=dataset_name,
    description="ADME RLM Stability experiment data released by the National Center for Advancing Translational Sciences",
    source="https://doi.org/10.1038/s41598-020-77327-0",
    annotations=annotations,
    tags=["ADME"],
    license="CC-BY-4.0",
    user_attributes={"year": "2020"},
    curation_reference="https://github.com/polaris-hub/polaris-recipes/org-Polaris/ncats_adme/RLM/01_ncats_rlm_data_curation.ipynb",
    owner=dict(user_id="polaris", slug="polaris"),
    readme=load_readme("org-Polaris/ncats_adme/RLM/ncats_rlm_readme.md")
)

In [9]:
# save the dataset to GCP
SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
dataset_path = dataset.to_json(SAVE_DIR)
dataset_path

'gs://polaris-public/polaris-recipes/org-polaris/ncats_adme/RLM/datasets/ncats_rlm-v1/dataset.json'

In [10]:
# upload to Polaris Hub
# dataset.upload_to_hub(owner=owner, access="private")